In [1]:
import numpy as np
import torch
from model_kitti_2012 import StereoNet
import utils as utils
import matplotlib.pyplot as plt
from pytorch_lightning.utilities.model_summary import ModelSummary
import stereonet_types as st
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import random_split
from torch import Generator

d:\__repos\engineering_thesis\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CHECKPOINT_PATH = r'D:\__repos\engineering_thesis\experiments\src\stereonet\lightning_logs\version_70\checkpoints\epoch=200-step=31155.ckpt'
KITTI_2012_ROOT = r'D:\engineering_thesis_data\Kitti_2012\data_stereo_flow\training'

In [3]:
model = StereoNet.load_from_checkpoint(CHECKPOINT_PATH)
model.eval()

val_transforms: st.TorchTransformers = [utils.Rescale()]
kitti_2012_dataset = utils.Kitti2012Dataset(KITTI_2012_ROOT, transforms=val_transforms)

train_len = int(0.8 * len(kitti_2012_dataset))
val_len = len(kitti_2012_dataset) - train_len
# print(len(kitti_2012_dataset), train_len, val_len)
train_dataset, val_dataset = random_split(
    kitti_2012_dataset, 
    [train_len, val_len],
    Generator().manual_seed(420))

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4, drop_last=False)

In [4]:
all_errors = np.empty(0)
print(all_errors.shape)

(0,)


In [12]:
for batch in val_loader:
# for batch in tqdm(val_loader):
    
    left = batch['left']
    right = batch['right']
    disp_gt = batch['disp_left']

    sample = {'left': left, 'right': right}

    with torch.no_grad():
        disp_pred = model(sample)

    # print(type(disp_pred_left))
    non_zero_pixels = torch.count_nonzero(disp_gt)
    print(non_zero_pixels)
    print(disp_gt.shape, disp_pred.shape)
    disp_pred = torch.where(disp_gt > 0, disp_pred, 0)
    print(disp_pred.max(), disp_pred.min())
    # print(non_zero_pixels)
    error_plane = torch.abs(disp_pred - disp_gt)
    print(error_plane.shape, error_plane.max(), error_plane.min(), np.count_nonzero(error_plane))
    # print(error_plane.shape)
    # print(torch.count_nonzero(error_plane))
    error_plane_filtered = \
        torch.where((error_plane > 3) & (error_plane > disp_gt * 0.05), error_plane, 0)
    print(error_plane_filtered.shape, error_plane_filtered.max(), error_plane_filtered.min(), np.count_nonzero(error_plane_filtered))
    print(torch.count_nonzero(error_plane_filtered))
    three_px_error = torch.count_nonzero(error_plane_filtered) / non_zero_pixels
    print(three_px_error)

    all_errors =  np.append(all_errors, error_plane_filtered.squeeze().detach().to('cpu').numpy())
    # print(all_errors.shape)
    print(three_px_error)
    # break

tensor(97322)
torch.Size([1, 1, 375, 1242]) torch.Size([1, 1, 375, 1242])
tensor(75.8574) tensor(0.)
torch.Size([1, 1, 375, 1242]) tensor(87.7953) tensor(0.) 97322
torch.Size([1, 1, 375, 1242]) tensor(87.7953) tensor(0.) 71346
tensor(71346)
tensor(0.7331)
tensor(0.7331)
tensor(110150)
torch.Size([1, 1, 376, 1241]) torch.Size([1, 1, 376, 1241])
tensor(70.6456) tensor(0.)
torch.Size([1, 1, 376, 1241]) tensor(76.8155) tensor(0.) 110150
torch.Size([1, 1, 376, 1241]) tensor(76.8155) tensor(0.) 77336
tensor(77336)
tensor(0.7021)
tensor(0.7021)
tensor(100923)
torch.Size([1, 1, 375, 1242]) torch.Size([1, 1, 375, 1242])
tensor(74.1692) tensor(0.)
torch.Size([1, 1, 375, 1242]) tensor(73.7925) tensor(0.) 100923
torch.Size([1, 1, 375, 1242]) tensor(73.7925) tensor(0.) 57786
tensor(57786)
tensor(0.5726)
tensor(0.5726)
tensor(149511)
torch.Size([1, 1, 376, 1241]) torch.Size([1, 1, 376, 1241])
tensor(68.2570) tensor(0.)
torch.Size([1, 1, 376, 1241]) tensor(25.5633) tensor(0.) 149511
torch.Size([1, 1,

In [6]:
print(batch['disp_left'].max(), batch['disp_left'].min())

tensor(95.2812) tensor(0.)


In [7]:
print(disp_pred.max(), disp_pred.min())

tensor(73.2382) tensor(0.)


In [8]:
print(all_errors.max(), all_errors.min())
print(len(all_errors))

156.8370361328125 0.0
18054068


In [9]:
# plt.bar(np.arange(len(all_errors)),all_errors)
# plt.show()